In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import json
import re

from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer

import matplotlib.pyplot as plt
import seaborn as sb


In [ ]:
# read in the metadata
df_meta = pd.read_csv('/kaggle/input/CORD-19-research-challenge/metadata.csv')
df_meta.head()

In [ ]:
# read in the json schema
with open('/kaggle/input/CORD-19-research-challenge/json_schema.txt') as open_json:
    json_schema = list(open_json)


In [ ]:
# read in the studies
studies_all = glob('/kaggle/input/CORD-19-research-challenge/**/*.json', recursive=True)
len(studies_all)


In [ ]:
# read in the first study and create dataframe for studies
with open(studies_all[0]) as file:
    first_study = json.load(file)
    
df_studies = pd.DataFrame.from_dict(first_study, orient='index').T
df_studies


In [ ]:
# add individual studies into one list
studies_list = []

for study in studies_all[1:]:
    df_temp = pd.read_json(study, orient='index').T
    df_studies = pd.concat([df_studies, df_temp], ignore_index=True, sort=False)
    

In [ ]:
# remove any unwanted column
df_studies.drop(columns=['back_matter', 'bib_entries', 'ref_entries'], inplace=True)


In [ ]:
# create new columns
df_studies['abstract_text'] = df_studies['abstract'].apply(lambda x: ','.join([i['text'] for i in x]) if x != [] else np.nan)
df_studies['title'] = df_studies['metadata'].apply(lambda x: x['title'] if (x != {} or x['title'] != '')  else np.nan)
df_studies['authors'] = df_studies['metadata'].apply(lambda x: x['authors'] if x != [] else np.nan)
df_studies['authors_list'] = df_studies['authors'].apply(lambda x: [' '.join([value if type(value) == str else 
                                                                    (value[0] if (len(value) > 0 and type(value) == list) 
                                                                    else (value+'; ' if key == 'last' else ''))
                                                                    for key, value in i.items()]).strip() for i in x]
                                                                    if x != [] else np.nan)
df_studies['full_text'] = df_studies['body_text'].apply(lambda x: ' '.join(['\n'.join([value if key == 'text' else ''
                                                                              for key, value in i.items()]) for i in x])
                                                                              if x != [] else np.nan)
